In [1]:
! pip uninstall spacy -y
!pip install spacy

Found existing installation: spacy 3.4.2
Uninstalling spacy-3.4.2:
  Successfully uninstalled spacy-3.4.2
  Using cached spacy-3.4.2-cp38-cp38-macosx_10_9_x86_64.whl (6.6 MB)


In [2]:
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 6.0 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.stem.lancaster import LancasterStemmer
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.porter import *
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA
from spacy.lang.en.stop_words import STOP_WORDS

In [2]:
nltk.download('maxent_ne_chunker')
from nltk.chunk import ne_chunk
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/riwasabri/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/riwasabri/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
with open('door_list.pickle', 'rb') as handle:
    df = pickle.load(handle)

### Pre-Processing

In [4]:
#Removing unnecessary columns

In [5]:
df.shape

(1050, 2)

In [6]:
sum([len(d.split(' ')) for d in df.content]) 

27521

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    1050 non-null   int64 
 1   content  1050 non-null   object
dtypes: int64(1), object(1)
memory usage: 16.5+ KB


In [8]:
df.columns

Index(['index', 'content'], dtype='object')

In [9]:
df['original']=df.content.copy()

In [10]:
df['original']

0        do you have any support for your customers i ...
1         hi our driver has been driving further and f...
2        your app and your site are both down for me i...
3         please add some validation in your driver ap...
4                                   never using  again   
                              ...                        
1045       would you consider covering fees temporaril...
1046    hey  since you are buying  you should help the...
1047    my experience with  and  reminds me why i pref...
1048     you make it very hard for users with a disabl...
1049     and  seriously and then i cancel and get cred...
Name: original, Length: 1050, dtype: object

### I am leaving a few unnecessary columns here that are going to help with cleaning out the text

In [11]:
#Removing 'quotedTweet' and 'retweetedTweet' and 'inReplyToTweetId' and 'cashtags' are not none to leave only original 
#and non original tweets 

### Pre-processing of text

In [12]:
def pre_process(text):
    #making text lowercase and deleting '\n','&amp'
    text= re.sub('(\n|&amp)',' ',text.lower())
    # removing hyperlinks
    text= re.sub('(http|ftp|https|www):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])','',text)
    #removing mentions
    text=re.sub('@[A-Za-z0-9-_]+','',text)
    text=re.sub('@[ ]+[A-Za-z0-9-_]+','',text)
    #removing hashtags
    text=re.sub('#[A-Za-z0-9-_]+','',text)
    #removing emojis and other character that are not alphanumerical
    text=re.sub("[^a-z\s\(\-:\)\\\/\];='#]",'',text)
    #words
    text=re.sub('night[A-Za-z0-9-_]+','',text)
    text=re.sub('sleep[A-Za-z0-9-_]+','',text)
    text=re.sub('[A-Za-z0-9-_]sleep','',text)
    text=re.sub('[A-Za-z0-9-_]night','',text)
    text=re.sub('dream[A-Za-z0-9-_]+','',text)
    text=re.sub('wake[A-Za-z0-9-_]+','',text)
    text=re.sub('think[A-Za-z0-9-_]+','',text)
    text=re.sub('wake','',text)
    text=re.sub('lmao','',text)
    text=re.sub('[A-Za-z0-9-_]wake','',text)
    text=re.sub('feel[A-Za-z0-9-_]','',text)
    text=re.sub('day[A-Za-z0-9-_]','',text)
    text=re.sub('today','',text)
    text=re.sub('terrible','',text)
    text=re.sub('try','',text)
    text=re.sub('hour','',text)
    text=re.sub('watch','',text)
    text=re.sub('come','',text)
    text=re.sub('say','',text)
    text=re.sub('said','',text)
    text=re.sub('year','',text)
    return text

In [13]:
df['content']=df.content.apply(lambda x: pre_process(x))

In [14]:
df.drop(columns=['id','outlinks','tcooutlinks','mentionedUsers','hashtags'],inplace=True)

KeyError: "['id' 'outlinks' 'tcooutlinks' 'mentionedUsers' 'hashtags'] not found in axis"

In [15]:
df.reset_index(inplace=True)

In [16]:
df=df.drop([893,894,907,925,928,931,937,938,943,954,956,958,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007])

### More text Processing: Lemmatization + Stopwords removal 

In [17]:
import spacy

In [18]:
nlp = spacy.load('en_core_web_sm')
stemmer= PorterStemmer()

In [19]:
def remove_common(sentence):
    list2=['hey','hello','caviar']
    new_word=[]
    for i in sentence.split(' '):
        if i in list2:
            new_word.append('')
        else:
            new_word.append(i)
    return ' '.join(new_word)

In [20]:
df['content']=df.content.apply(lambda x:remove_common(x))

In [21]:
for doc in df['content']:
    if 'today' in doc:
        print (doc)

In [22]:
docs = list(nlp.pipe(df.content))

In [23]:
nlp.Defaults.stop_words |= {"nightmare","nightmares",'night','dream','sleep','asleep','rembember','fucking','fuck','remembered'}

In [24]:
docs_clean = [[w.lemma_ for w in doc if (not w.is_stop and not w.is_punct and not w.like_num and len(w)>2)\
               or (w.lemma_=='not')] for doc in docs]
df['content_clean'] = docs_clean

In [25]:
docs_list_clean = [' '.join(map(str,doc)) for doc in docs_clean]

In [26]:
docs_list_clean

['support customer spend food restaurant not exist dispute charge',
 'driver drive away not pick order drive minute not chat support cancel reorder',
 'app site order process idea way way check way contact help',
 'add validation driver app check near delivery address not not let mark order deliver like people constantly mistake app hassle',
 '',
 'angry give wrong order not refund credit not fault get wrong order',
 'app',
 'like site have trouble',
 'delivery',
 'access list nice restaurant treat promo',
 'huh not reachable ssl not fortunate happen',
 'wrong order     way contact app site crash and/or not load file complaint',
 'go not credit restaurant leave order option accept credit not get kid',
 'ummhow cancel order not show not track not find delivery person pick restaurant close',
 'completely break app drop cart log tip option web app card verification system break error connect paypal restaurant place order lose business',
 'not way remove item cart start new order diff rest

In [27]:
df.original[0]

' do you have any support for your customers i spent  on food from a restaurant that doesnt seem to exist should i just dispute my charge'

In [60]:
cv = TfidfVectorizer(stop_words='english',min_df=10,max_df=0.80)
X = cv.fit_transform(docs_list_clean)

df_X = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

In [77]:
nmf = NMF(n_components=7)
nmf.fit(df_X)
nmf

/Users/riwasabri/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


NMF(n_components=7)

In [49]:
pca = PCA(n_components=8)
pca.fit(df_X)
nmf

NameError: name 'nmf' is not defined

In [50]:
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [51]:
display_topics(pca, cv.get_feature_names(), 5)


Topic  1
order, food, deliver, delivery, restaurant

Topic  2
app, order, work, place, website

Topic  3
order, help, account, need, place

Topic  4
help, app, restaurant, delivery, need

Topic  5
food, deliver, account, driver, help

Topic  6
service, customer, help, bad, experience

Topic  7
work, time, need, issue, code

Topic  8
restaurant, support, time, ago, wait




(PCA(n_components=8),
 ['able',
  'account',
  'add',
  'address',
  'ago',
  'answer',
  'app',
  'area',
  'arrive',
  'asap',
  'ask',
  'available',
  'away',
  'bad',
  'business',
  'cancel',
  'card',
  'care',
  'change',
  'charge',
  'chat',
  'check',
  'close',
  'code',
  'cold',
  'company',
  'completely',
  'confirm',
  'contact',
  'courier',
  'credit',
  'customer',
  'day',
  'deactivate',
  'delay',
  'deliver',
  'delivery',
  'dinner',
  'doordash',
  'drive',
  'driver',
  'eat',
  'email',
  'end',
  'error',
  'experience',
  'fee',
  'fix',
  'food',
  'form',
  'fraud',
  'free',
  'friend',
  'good',
  'got',
  'guy',
  'hack',
  'half',
  'happen',
  'hear',
  'help',
  'hungry',
  'ing',
  'instead',
  'issue',
  'item',
  'know',
  'late',
  'later',
  'leave',
  'let',
  'like',
  'link',
  'list',
  'live',
  'lock',
  'log',
  'login',
  'long',
  'look',
  'lose',
  'lunch',
  'mark',
  'meal',
  'message',
  'min',
  'minute',
  'miss',
  'money',
 

In [78]:
display_topics(nmf, cv.get_feature_names(), 5)


Topic  1
order, place, cancel, receive, issue

Topic  2
account, email, support, deactivate, contact

Topic  3
app, work, website, log, site

Topic  4
help, need, restaurant, work, business

Topic  5
service, customer, bad, experience, support

Topic  6
food, deliver, driver, minute, arrive

Topic  7
delivery, time, restaurant, fee, charge




(NMF(n_components=7),
 ['able',
  'account',
  'add',
  'address',
  'ago',
  'answer',
  'app',
  'area',
  'arrive',
  'asap',
  'ask',
  'available',
  'away',
  'bad',
  'business',
  'cancel',
  'card',
  'care',
  'change',
  'charge',
  'chat',
  'check',
  'close',
  'code',
  'cold',
  'company',
  'completely',
  'confirm',
  'contact',
  'courier',
  'credit',
  'customer',
  'day',
  'deactivate',
  'delay',
  'deliver',
  'delivery',
  'dinner',
  'doordash',
  'drive',
  'driver',
  'eat',
  'email',
  'end',
  'error',
  'experience',
  'fee',
  'fix',
  'food',
  'form',
  'fraud',
  'free',
  'friend',
  'good',
  'got',
  'guy',
  'hack',
  'half',
  'happen',
  'hear',
  'help',
  'hungry',
  'ing',
  'instead',
  'issue',
  'item',
  'know',
  'late',
  'later',
  'leave',
  'let',
  'like',
  'link',
  'list',
  'live',
  'lock',
  'log',
  'login',
  'long',
  'look',
  'lose',
  'lunch',
  'mark',
  'meal',
  'message',
  'min',
  'minute',
  'miss',
  'money',
 

In [52]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity


In [53]:
new_coords=pca.transform(df_X)

In [54]:
def top_3_similar(index):
    
    dist=pairwise_distances(new_coords[index].reshape(1,-1),new_coords,metric='cosine')
    indices=(dist.argsort().tolist()[0][0:4])
    print ('Tweet: '+ df.original[index])
    print('Recommended Tweets:')
    print('---------------')
    for i in indices:
        print(i,df.original[i])
        print('---------------')
   

In [55]:
top_3_similar(2)

Tweet:  your app and your site are both down for me i have an order in process and i have no idea if its on its way no way to check and no way to contact help
Recommended Tweets:
---------------
2  your app and your site are both down for me i have an order in process and i have no idea if its on its way no way to check and no way to contact help
---------------
52  i got a notification that if i updated your app id get  off my order so i did it and now it says im not eligible can you please help
---------------
100  order has not arrived and im locked out of app help
---------------
449  trying to confirm an order that was made  min ago in-app chat not working please help
---------------


In [56]:
top_3_similar(1)

Tweet:   hi our driver has been driving further and further away and hasn't even picked up the order they've been driving around for  minutes and we can't get your chat support to cancel and reorder for us 
Recommended Tweets:
---------------
1   hi our driver has been driving further and further away and hasn't even picked up the order they've been driving around for  minutes and we can't get your chat support to cancel and reorder for us 
---------------
883    ive been waiting for a dasher to be assigned to my caviar order for more than an hour and a half and have been waiting for a support agent for over  minutes whats up
---------------
826  ordered fastbite  minutes ago and the courier is nowhere near my apt you're also not responding to texts convenient
---------------
71  comon guys ordered hrs a go no food no status cant get a refund because support has been silent for hrs
---------------


In [64]:
#LDA
import gensim
from gensim import corpora, models, similarities, matutils

In [65]:
doc_word = cv.fit_transform(docs_list_clean).transpose()

In [68]:
pd.DataFrame(doc_word.toarray(), cv.get_feature_names())

,0,1,2,3,4,5,6,7,8,9,...,981,982,983,984,985,986,987,988,989,990
able,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
account,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.168935,0.0
add,0.0,0.0,0.0,0.300387,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
address,0.0,0.0,0.0,0.291126,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ago,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
website,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
week,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
work,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.530393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
wrong,0.0,0.0,0.0,0.000000,0.0,0.779168,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [73]:
corpus = matutils.Sparse2Corpus(doc_word)

id2word = dict((v, k) for k, v in cv.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=100)

In [76]:
lda.print_topics()

[(0,
  '0.089*"food" + 0.073*"delivery" + 0.068*"restaurant" + 0.066*"deliver" + 0.055*"order" + 0.047*"driver" + 0.047*"help" + 0.039*"fee" + 0.035*"minute" + 0.030*"charge"'),
 (1,
  '0.109*"app" + 0.068*"pay" + 0.064*"order" + 0.058*"issue" + 0.051*"dinner" + 0.047*"guy" + 0.044*"suppose" + 0.042*"credit" + 0.040*"happen" + 0.036*"friend"'),
 (2,
  '0.091*"service" + 0.064*"customer" + 0.058*"work" + 0.057*"time" + 0.055*"bad" + 0.044*"code" + 0.040*"experience" + 0.036*"use" + 0.032*"issue" + 0.027*"order"'),
 (3,
  '0.079*"email" + 0.060*"thank" + 0.054*"address" + 0.051*"change" + 0.049*"support" + 0.042*"chat" + 0.041*"min" + 0.040*"send" + 0.039*"contact" + 0.037*"respond"'),
 (4,
  '0.078*"add" + 0.064*"courier" + 0.049*"let" + 0.045*"area" + 0.045*"app" + 0.037*"reply" + 0.037*"drive" + 0.036*"tip" + 0.036*"away" + 0.035*"lunch"'),
 (5,
  '0.058*"think" + 0.053*"super" + 0.052*"hungry" + 0.049*"instead" + 0.047*"refund" + 0.046*"doordash" + 0.043*"fraud" + 0.043*"care" + 0.04